
## 贝叶斯回归
贝叶斯回归是一种基于贝叶斯统计推断的回归方法。它通过引入先验分布来表达对参数的不确定性，并利用观测数据来更新参数的后验分布。假设我们有一个训练集包含$N$个样本，每个样本由输入特征$X$和对应的输出标签$y$组成。要经过的步骤是参数建模 -> 后验推断 -> 参数估计和预测。

优缺点：
* 贝叶斯回归引入先验知识。适用于小样本、高噪声以及需要考虑模型不确定性的数据。
* 计算复杂度较高，需要进行概率推断和参数估计，不适用于高维数据。

下面是使用Python和PyMC3库实现贝叶斯线性回归的示例代码：

In [2]:
import pymc3 as pm
import numpy as np

# 构造训练集
X = np.random.randn(100, 2)
w_true = np.array([3, 5])
y = X.dot(w_true) + np.random.randn(100)

# 创建贝叶斯模型
with pm.Model() as model:
    # 定义参数的先验分布
    w = pm.Normal('w', mu=0, sd=1, shape=2)
    sigma = pm.HalfNormal('sigma', sd=1)
    
    # 定义输出标签的条件分布
    y_obs = pm.Normal('y_obs', mu=pm.math.dot(X, w), sd=sigma, observed=y)
    
    # 进行后验推断
    trace = pm.sample(1000, tune=1000)

# 输出参数估计结果
print("参数估计结果：")
print(pm.summary(trace)['mean'])

# 进行预测
x_new = np.array([[1, 2], [3, 4]])  # 新的输入特征
with model:
    post_pred = pm.sample_posterior_predictive(trace, samples=1000, vars=[y_obs])
    y_pred_mean = np.mean(post_pred['y_obs'], axis=0)

print("预测结果：")
print(y_pred_mean)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, w]
Sampling 4 chains: 100%|██████████| 8000/8000 [00:02<00:00, 3036.46draws/s]


参数估计结果：
w__0     2.890521
w__1     4.865710
sigma    0.945172
Name: mean, dtype: float64


100%|██████████| 1000/1000 [00:01<00:00, 503.68it/s]

预测结果：
[  1.03153314  -3.61562589   2.16812673  -3.11588044   9.60127944
  12.16905026  -0.23648124   4.25748233  -4.45208439  -7.32287496
   3.27508941   6.5172929   -0.836755     2.46292051  -6.08647484
   0.58920231  -3.83147137   6.42264627  -3.61320578   1.41008255
   6.48528022   0.24595292  -0.12118109   0.07075903  -8.62226662
  11.63699736   3.94074076  -0.54817445   1.58446021   0.80333973
  -8.5854543   -2.80430425  -5.25514791  -4.58258886 -14.3224442
  -1.84326201   4.46406574   6.46984307   0.21513803 -14.46026694
  -2.29835858   0.19768268   2.16854864  -4.15003522   9.75786452
  -3.88164949  -7.4700798    3.36505357   4.478443     1.69431611
  -2.70376263   7.05479675   1.63607418  -4.06979375  -4.62857722
  -2.06437145   5.40670935   3.61310883   3.05419556  -1.92042677
  -3.81055859  -0.7341311   -2.46628125   0.46042887   6.3505802
  -5.52622373  -3.90441137  -4.68569514   2.0306271    0.6378601
   2.38703032  -2.07502364   4.05460138   6.58315489  -0.94999262
   3.89

### 贝叶斯岭回归

贝叶斯岭回归（Bayesian Ridge Regression）是一种基于贝叶斯统计推断的回归方法，结合了岭回归和贝叶斯推断的思想。

具体步骤如下：

----------
假设模型为：

$y = Xw + ε$ 其中，$y$是目标变量，$X$是特征矩阵，$w$是待估计的参数向量，$ε$是误差项。

输入：特征矩阵 $X$（大小为 $n \times p$），目标变量向量 $y$（长度为 $n$）

输出：参数估计值 $w$

----------


* 定义先验分布：假设参数 $w$ 的先验分布为 $p(w)$。

* 计算后验分布：根据贝叶斯定理，可以得到参数 $w$ 的后验分布：

    $ p(w|X, y) = \frac{p(y|X, w) \cdot p(w)}{p(y|X)} $

    其中，$p(y|X, w)$ 是给定参数 $w$ 条件下观测数据 $y$ 的概率密度函数，$p(w)$ 是参数 $w$ 的先验分布，$p(y|X)$ 是边际似然函数。

* 参数估计：通过后验分布可以获得参数的点估计或分布估计。

    * 点估计：例如，使用后验分布的均值作为参数的点估计：

    $ \hat{w} = E[w|X, y] = \int w \cdot p(w|X, y) dw $

    * 分布估计：通过后验分布得到参数的概率分布，用于描述参数的不确定性：

    $ p(w|X, y) $

通过对后验分布进行采样或使用解析的数学表达式，可以获得对参数 $w$ 的估计结果，其中包括点估计和不确定性估计。

---------

下面是使用Python和Scikit-learn库实现贝叶斯岭回归的示例代码：

In [1]:
from sklearn import linear_model

X = [[0., 0.], [1., 1.], [2., 2.], [3., 3.]]
Y = [0., 1., 2., 3.]

# 创建贝叶斯岭回归模型
reg = linear_model.BayesianRidge()

# 拟合模型
reg.fit(X, Y)

# 输出参数估计结果
print("参数估计结果：")
print(reg.coef_)

# 进行预测
y_pred = reg.predict([[1, 0.]])


参数估计结果：
[0.49999993 0.49999993]


## 主动相关决策理论 - ARD

ARDRegression类似于贝叶斯岭回归（Bayesian Ridge Regression），但具有更强的稀疏性。这是因为ARDRegression引入了不同于贝叶斯岭回归的先验假设，即权重 $w$ 的分布不再是球形的高斯分布，而是轴对齐的椭圆高斯分布。

ARDRegression中的每个权重 $wi$ 都有一个单独的标准差 $λ_i$ 。所有 $λ_i$ 的先验分布由超参数 $λ1$ 、$λ2$ 等确定，通常使用相同的 $\gamma$ 分布。

ARDRegression可以用于特征选择，因为它倾向于将不相关或弱相关的特征的权重设为0，从而实现了稀疏性。这使得模型更容易解释，并且可以提高泛化性能。

不过ARD也比较慢。

下面给个sklearn的例子：

In [2]:
from sklearn.datasets import make_regression
from sklearn.linear_model import ARDRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 生成随机回归数据集
X, y = make_regression(n_samples=100, n_features=10, noise=0.1, random_state=42)

# 将数据集拆分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 创建并拟合ARDRegression模型
model = ARDRegression()
model.fit(X_train, y_train)

# 使用模型进行预测
y_pred = model.predict(X_test)

# 计算均方误差（Mean Squared Error）
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error: ", mse)

Mean Squared Error:  0.010270127022996813


## 参考
* 贝叶斯回归：使用 PyMC3 实现贝叶斯回归 https://baijiahao.baidu.com/s?id=1745363357212714786&wfr=spider&for=pc